# Covid-19 en Centroamérica
> Dashboard

- toc: false
- branch: master
- badges: true
- comments: false
- image: images/covid-19-ca.png
- permalink: /covid-19-ca/
- author: "Libre AI"
- categories: [centroamerica, covid19]

> Última actualización 2020-03-28 10:24 AM CST

In [2]:
#hide
import math
from IPython.display import display, HTML

In [3]:
#hide

#latest case count
cases_by_country = {'gt': 32, 'bz': 2, 'hn': 95, 'sv': 19, 'ni': 4, 'cr': 263, 'pa': 786}

In [4]:
#hide

html_head = '''
<html>
<head>
  <title>Covid-19 en Centroamérica</title>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0">

  <link rel="shortcut icon" type="image/x-icon" href="docs/images/favicon.ico" />

    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.6.0/dist/leaflet.css" integrity="sha512-xwE/Az9zrjBIphAcBb3F6JVqxf46+CDLwfLMHloNu6KEQCAWi6HcDUbeOfBIptF7tcCzusKFjFw2yuvEpDL9wQ==" crossorigin=""/>
    <script src="https://unpkg.com/leaflet@1.6.0/dist/leaflet.js" integrity="sha512-gZwIG9x3wUXg2hdXF6+rVkLF/0Vi9U8D2Ntg4Ga5I5BZpVkVxlJWbSQtXPSiUTtC0TjtGOmxa1AJPuV0CPthew==" crossorigin=""></script>
</head>
<body>

<div id="covid10-map" style="width: 800px; height: 600px;"></div>
'''

html_foot = '''
</script>

</body>
</html>
'''

In [5]:
#hide

map_center_lat, map_center_long = (12.5, -87)
zoom = 6
max_zoom = 7

In [6]:
#hide

base_map_html = '''
<script>
  var mymap = L.map('covid10-map').setView([{}, {}], {});
'''.format(map_center_lat, map_center_long, zoom)

# base_layer = '''
# L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
#     attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
#     subdomains: 'abcd',
#     maxZoom: {}
# }}).addTo(mymap);
# '''.format(max_zoom)

# base_layer = '''
# L.tileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{{z}}/{{y}}/{{x}}', {{
#     attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
#     maxZoom: {}
# }}).addTo(mymap);
# '''.format(max_zoom)

base_layer = '''
L.tileLayer('https://tiles.stadiamaps.com/tiles/osm_bright/{{z}}/{{x}}/{{y}}{{r}}.png', {{
    attribution: '&copy; <a href="https://stadiamaps.com/">Stadia Maps</a>, &copy; <a href="https://openmaptiles.org/">OpenMapTiles</a> &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors',
    maxZoom: {}
}}).addTo(mymap);
'''.format(max_zoom)

In [7]:
#hide

class Country:
    def __init__(self, name, lat, long):
        super()
        self.cases = 0
        self.name = name
        self.lat = lat
        self.long = long


In [8]:
#hide

gt = Country('Guatemala', 14.63, -90.56)
bz = Country('Belize', 17.25, -88.80)
hn = Country('Honduras', 14.08, -87.24)
sv = Country('El Salvador', 13.69, -89.25)
ni = Country('Nicaragua', 12.10, -86.33)
cr = Country('Costa Rica', 9.94, -84.15)
pa = Country('Panama', 9.08, -79.59)

countries = {'gt': gt, 'bz': bz, 'hn': hn, 'sv': sv, 'ni': ni, 'cr': cr, 'pa': pa}

In [9]:
#hide

# set latest cases
for c in countries:
    countries[c].cases = cases_by_country[c]

In [10]:
#hide

map_circles_html = []
for c in countries:
    c_circle = '''
        L.circle([{}, {}], {}*3000, {{
          color: 'red',
          fillColor: '#f03',
          fillOpacity: 0.5
          }}).addTo(mymap)
          .bindTooltip("<div><b>{}</b><br/><span style='color:red'><h3 style='display:inline'>{}</h3></span>&nbsp;personas infectadas</div>", {{sticky: true}});'''.format(countries[c].lat,
                                                                               countries[c].long,
                                                                               math.sqrt(countries[c].cases),
                                                                               countries[c].name,
                                                                               countries[c].cases)
    
    map_circles_html.append(c_circle)
    

In [11]:
#hide
display(HTML(html_head + base_map_html + base_layer + "\n".join(map_circles_html)))

In [13]:
#hide_input

import folium


m = folium.Map(location=[map_center_lat, map_center_long],
               zoom_start=zoom
              )

folium.CircleMarker(
    location=[14.63, -90.56],
    radius=math.sqrt(32),
    color='red',
    fill=True,
    fill_color='#f03',
    tooltip="<div><b>Guatemala</b><br/><span style=\'color:red\'><h3 style=\'display:inline\'>32</h3></span>&nbsp;personas infectadas</div>"
).add_to(m)

m
